In [0]:
!nvidia-smi

!pip install pytorch-nlp
!pip install pytorch-pretrained-bert



Sun Jul  7 13:34:02 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Get Data from Dropbox
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
%cd Dropbox-Uploader
!chmod +x dropbox_uploader.sh
!bash dropbox_uploader.sh
!echo "CHTZbyeOjyYAAAAAAAAAfZTfC_d3kTmHXSqg7-941o9Nolj-FEAJbjyUKutEtCSu" > token.txt
!bash dropbox_uploader.sh

!bash dropbox_uploader.sh download cola.zip
!unzip cola.zip
!rm cola.zip

Cloning into 'Dropbox-Uploader'...
remote: Enumerating objects: 951, done.
remote: Total 951 (delta 0), reused 0 (delta 0), pack-reused 951
Receiving objects: 100% (951/951), 309.19 KiB | 1.09 MiB/s, done.
Resolving deltas: 100% (505/505), done.
/content/Dropbox-Uploader

 This is the first time you run this script, please follow the instructions:

 1) Open the following URL in your Browser, and log in using your account: https://www.dropbox.com/developers/apps
 2) Click on "Create App", then select "Dropbox API app"
 3) Now go on with the configuration, choosing the app permissions and access restrictions to your DropBox folder
 4) Enter the "App Name" that you prefer (e.g. MyUploader11562246907104)

 Now, click on the "Create App" button.

 When your new App is successfully created, please click on the Generate button
 under the 'Generated access token' section, then execute the following command:

 echo "INPUT_YOUR_ACCESS_TOKEN_HERE" > token.txt


 This is the first time you run thi

In [0]:


import numpy as np
import torch.nn as nn

#from torchnlp.datasets import smt_dataset
#from torchnlp.datasets import imdb_dataset


import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForPreTraining, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from random import Random
torch.set_default_tensor_type('torch.cuda.FloatTensor')


In [0]:
#DATA PREPROCESSING FUNCTIONS

#FUNCTION TO PAD DATA TO UNIFORM LENGTH
def padding(sequences, maxlen=None, dtype=np.int, padding='post', value=0.):
     
    # padding = "pre" pads from the front, "post" pads at the end
    # Function pads to convert lists of strings (of uneven length) into numpy arrays
  
    lengths = [len(s) for s in sequences]

    nb_samples = len(sequences)
    
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    padded = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if padding == 'post':
            padded[idx, :len(s)] = s
        elif padding == 'pre':
            padded[idx, -len(s):] = s
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return padded
  
#Function to pre-process data for BERT
def process_bert(data,num_train,max_len):
  
  #Import Bert tokenizer
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  
  #Create arrays to store our tokenised training data
  tokens = []
  tokens_ids = []
  segment_ids = []
  input_masks = []
  
  #Tokenise the training data, add CLS/SEP tokens and then convert to IDs
  for i in range(num_train):

    #Tokenize text and add CLS/SEP tokens, Truncate if over 510 tokens
    text = data[i]
    token = tokenizer.tokenize(text)
    token_len = len(token)
    if(token_len)>510:
      token = token[-max_len:]
    token.insert(0,'[CLS]')
    token.append('[SEP]')

    #Create segment IDs and inputs masks
    segment_id = [0] * len(token)
    input_mask = [1] * len(token)
    segment_ids.append(segment_id)
    input_masks.append(input_mask)

    #Convert tokens to IDs
    token_id = tokenizer.convert_tokens_to_ids(token)
    tokens_ids.append(token_id)
  

  #Pad the training data so everything is of uniform length  
  tokens_ids = padding(tokens_ids)
  segment_ids = padding(segment_ids)
  input_masks = padding(input_masks)

  max_length = len(tokens_ids[0])


  #Convert training data and labels to tensors
  ids_tensor = torch.tensor([tokens_ids], dtype=torch.long).resize_((num_train,max_length))
  segment_ids_tensor = torch.tensor([segment_ids], dtype=torch.long).resize_((num_train,max_length))
  input_masks_tensor = torch.tensor([input_masks], dtype=torch.long).resize_((num_train,max_length))
  
  return ids_tensor, segment_ids_tensor, input_masks_tensor

#FUNCTION TO CREATE LABELS FOR SMT
def create_SMT_labels(data,num_train):
  labels_tensor = torch.empty(num_train, dtype=torch.long).random_(3)
  labels = data.__getitem__('label')
  labels = labels[0:num_train]

  pos_indices = [i for i, x in enumerate(labels) if x == "positive"]
  neut_indices = [i for i, x in enumerate(labels) if x == "neutral"]
  neg_indices = [i for i, x in enumerate(labels) if x == "negative"]

  labels_tensor[pos_indices] = 0
  labels_tensor[neut_indices] = 1
  labels_tensor[neg_indices] = 2
  
  return labels_tensor

#FUNCTION TO CREATE LABELS FOR IMDB
def create_COLA_labels(data,num_train):
  labels_tensor = torch.empty(num_train, dtype=torch.long).random_(2)
  labels = data
  labels = labels[0:num_train]

  pos_indices = [i for i, x in enumerate(labels) if x == "1"]
  neg_indices = [i for i, x in enumerate(labels) if x == "0"]

  labels_tensor[pos_indices] = 1
  labels_tensor[neg_indices] = 0
  
  return labels_tensor

#FUNCTION TO RESTRICT TRAINING DATASET
def restrict_data_func(train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, num_each_label, num_labels, train_labels_tensor_):
  train_ids_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_segment_ids_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_input_masks_tensor = torch.empty((num_each_label*num_labels,max_length), dtype=torch.long).random_(num_labels)
  train_labels_tensor = torch.empty(num_each_label*num_labels, dtype=torch.long).random_(num_labels)
  for i in range(num_labels):
  
    train_ids_tensor[i*num_each_label:(i+1)*num_each_label] = train_ids_tensor_[train_labels_tensor_==i][:num_each_label]
    train_segment_ids_tensor[i*num_each_label:(i+1)*num_each_label] = train_segment_ids_tensor_[train_labels_tensor_==i][:num_each_label]
    train_input_masks_tensor[i*num_each_label:(i+1)*num_each_label] = train_input_masks_tensor_[train_labels_tensor_==i][:num_each_label] 
 
  for i in range(num_labels):
    train_labels_tensor[i*num_each_label:(i+1)*num_each_label]=train_labels_tensor_[train_labels_tensor_==i][:num_each_label]

  return train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor

#Function to shuffle a tensor
def shuffle_tensor(tensor):
  torch.manual_seed(25082019)
  print(torch.randperm(tensor.shape[0]))
  tensor = tensor[torch.randperm(tensor.shape[0])]
  print(torch.randperm(tensor.shape[0]))
  print(len(torch.randperm(tensor.shape[0])))
  return tensor

#Import dataset
#train = smt_dataset(train=True)
#valid = smt_dataset(dev=True)
#test = smt_dataset(test=True)

#train = imdb_dataset(train=True)
#test = imdb_dataset(test=True)



In [0]:
# import model for fine-tuning
num_labels = 2
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
model.cuda()


100%|██████████| 407873900/407873900 [00:15<00:00, 26649367.36B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [0]:
#Load preprocessed Data

train_labels = np.load('cola/cola_train_labels.npy')
train_text = np.load('cola/cola_train_text.npy')
valid_labels_in = np.load('cola/cola_in_domain_dev_labels.npy')
valid_text_in = np.load('cola/cola_in_domain_dev_text.npy')
valid_labels_out = np.load('cola/cola_out_domain_dev_labels.npy')
valid_text_out = np.load('cola/cola_out_domain_dev_text.npy')

print(len(valid_text_out))

516


In [0]:
#DATA PREP

num_train = 8550
num_valid_in = 527
num_valid_out = 516
torch.manual_seed(25082019)


max_len = 510
train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_ = process_bert(train_text,num_train,max_len)
valid_ids_tensor, valid_segment_ids_tensor, valid_input_masks_tensor = process_bert(valid_text_in,num_valid_in,max_len)
valid_ids_tensor_out, valid_segment_ids_tensor_out, valid_input_masks_tensor_out = process_bert(valid_text_out,num_valid_out,max_len)


max_length = train_ids_tensor_.shape[1]

print(max_length)

print(valid_ids_tensor.shape[1])
print(valid_ids_tensor_out.shape[1])
#test_ids_tensor, test_segment_ids_tensor, test_input_masks_tensor = process_bert(test,num_test,max_len)
#train_labels_tensor_ = create_IMDB_labels(train,num_train)
#test_labels_tensor = create_IMDB_labels(test,num_test)

train_labels_tensor_ = create_COLA_labels(train_labels,num_train)
valid_labels_tensor = create_COLA_labels(valid_labels_in,num_valid_in)
valid_labels_tensor_out = create_COLA_labels(valid_labels_out,num_valid_out)


"""
#Set seed so that shuffle is the same for each tensor

#Randomly shuffle training data to get mix of both classes
train_ids_tensor_ = shuffle_tensor(train_ids_tensor_)
train_segment_ids_tensor_ = shuffle_tensor(train_segment_ids_tensor_)
train_input_masks_tensor_ = shuffle_tensor(train_input_masks_tensor_)
train_labels_tensor_ = shuffle_tensor(train_labels_tensor_)


#Split training data into training and validation
num_valid = 2500
valid_ids_tensor = train_ids_tensor_[0:num_valid]
valid_segment_ids_tensor = train_segment_ids_tensor_[0:num_valid]
valid_input_masks_tensor = train_input_masks_tensor_[0:num_valid]
valid_labels_tensor = train_labels_tensor_[0:num_valid]

train_ids_tensor_ = train_ids_tensor_[num_valid:]
train_segment_ids_tensor_ = train_segment_ids_tensor_[num_valid:]
train_input_masks_tensor_= train_input_masks_tensor_[num_valid:]
train_labels_tensor_ = train_labels_tensor_[num_valid:]

"""



#DECIDE HOW MANY OF EACH CLASS TO USE IN TRAINING DATA
restrict_training_data = 0 #Set equal to 0 or 1 
num_each_label=10

if restrict_training_data:
  train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor = restrict_data_func(train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, num_each_label, num_labels,train_labels_tensor_ )
else:
  train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor = train_ids_tensor_, train_segment_ids_tensor_, train_input_masks_tensor_, train_labels_tensor_


train_max_batch = 10
valid_max_batch = 17
valid_max_batch_out = 12
test_max_batch=10

#Load data into dataloader
train_data = TensorDataset(train_ids_tensor, train_segment_ids_tensor, train_input_masks_tensor, train_labels_tensor)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_max_batch)

valid_data = TensorDataset(valid_ids_tensor, valid_segment_ids_tensor, valid_input_masks_tensor, valid_labels_tensor)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=valid_max_batch)

valid_data_out = TensorDataset(valid_ids_tensor_out, valid_segment_ids_tensor_out, valid_input_masks_tensor_out, valid_labels_tensor_out)
valid_sampler_out = SequentialSampler(valid_data_out)
valid_dataloader_out = DataLoader(valid_data_out, sampler=valid_sampler_out, batch_size=valid_max_batch_out)

#test_data = TensorDataset(test_ids_tensor, test_segment_ids_tensor, test_input_masks_tensor, test_labels_tensor)
#test_sampler = SequentialSampler(test_data)
#test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=test_max_batch)




100%|██████████| 231508/231508 [00:00<00:00, 933360.37B/s]


47
31
35


In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    
optimizer = BertAdam(optimizer_grouped_parameters, lr=3e-5)

t_total value of -1 results in schedule not being applied


In [0]:
#Training
#Set Loss Function
cross_entropy = nn.CrossEntropyLoss()

max_grad_norm = 1.0
# TRAIN loop
model.train()
tr_loss = 0
nb_tr_steps = 0
epochs = 300


for j in range(epochs):
  print("epoch: ", j+1)
  
  print("Begin Training")
  
  for step, batch in enumerate(train_dataloader):
      
      if(step%25==0):
        print("batch: ", step+1)

      # add batch to gpu
      #batch = tuple(t.to(device) for t in batch)
      
      b_train_ids, b_segment_ids, b_input_masks, b_train_labels = batch
      
      
      # forward pass
      logits = model(b_train_ids,b_segment_ids,b_input_masks,labels=None)
      loss = cross_entropy(logits,b_train_labels)
      
      #Calculate accuracy
      
      
      # backward pass
      loss.backward()

      # track train loss
      tr_loss += loss.item()
      nb_tr_steps += 1

      # gradient clipping
      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

      # update parameters
      optimizer.step()
      model.zero_grad()

      
  # print train loss per epoch
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
  
  
  print("Begin Validation")

  model.eval()
  total_correct = 0
  for step, batch in enumerate(valid_dataloader):

    if(step%10==0):
      print("batch: ", step+1)

    b_valid_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

    with torch.no_grad():
      logits = model(b_valid_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)


    a = logits.argmax(1)
    b = b_valid_labels

    temp_correct = torch.sum(a==b).cpu().numpy()
    total_correct+=temp_correct
  print("Validation In Domain Acccuracy:")
  print(total_correct/num_valid_in)
  
  model.eval()
  total_correct = 0
  for step, batch in enumerate(valid_dataloader_out):

    if(step%10==0):
      print("batch: ", step+1)

    b_valid_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

    with torch.no_grad():
      logits = model(b_valid_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)


    a = logits.argmax(1)
    b = b_valid_labels

    temp_correct = torch.sum(a==b).cpu().numpy()
    total_correct+=temp_correct
  print("Validation Out Domain Acccuracy:")
  print(total_correct/num_valid_out)
  
  

    
            
  

epoch:  1
Begin Training
batch:  1
batch:  26
batch:  51
batch:  76
batch:  101
batch:  126
batch:  151
batch:  176
batch:  201
batch:  226
batch:  251
batch:  276
batch:  301
batch:  326
batch:  351
batch:  376
batch:  401
batch:  426
batch:  451
batch:  476
batch:  501
batch:  526
batch:  551
batch:  576
batch:  601
batch:  626
batch:  651
batch:  676
batch:  701
batch:  726
batch:  751
batch:  776
batch:  801
batch:  826
batch:  851
Train loss: 0.5589803475908368
Begin Validation
batch:  1
batch:  11
batch:  21
batch:  31
Validation In Domain Acccuracy:
0.7874762808349146
batch:  1
batch:  11
batch:  21
batch:  31
batch:  41
Validation Out Domain Acccuracy:
0.7461240310077519
epoch:  2
Begin Training
batch:  1
batch:  26
batch:  51
batch:  76


In [0]:
"""
model.eval()
total_correct = 0
for step, batch in enumerate(test_dataloader):

  if(step%25==0):
    print("batch: ", step+1)

  b_test_ids, b_segment_ids, b_input_masks, b_valid_labels = batch

  with torch.no_grad():
    logits = model(b_test_ids, token_type_ids=b_segment_ids, attention_mask=b_input_masks)

  a = logits.argmax(1)
  b = b_valid_labels

  temp_correct = torch.sum(a==b).cpu().numpy()
  total_correct+=temp_correct
print("Test Acccuracy:")
print(total_correct/num_test)

"""